In [ ]:
from tensorflow.keras import models, layers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from functools import partial
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:
def create_model(optimizer, activation, padding, pooling, learning_rate, batch_normalization, dropout):
    
    model = models.Sequential()
    DefaultConv2D = partial(layers.Conv2D, kernel_size=(3,3), activation=activation, padding=padding)

    # First Convolution Layer
    model.add(DefaultConv2D(filters=32, input_shape=(X_gadf_train[0].shape[0], 
                                                     X_gadf_train[0].shape[1], 
                                                     X_gadf_train[0].shape[2])))
    # Batch Nomalization
    if batch_normalization==True:
        model.add(layers.BatchNormalization())
    
    # Pooling Layer
    if pooling=="max":
        model.add(layers.MaxPooling2D((2,2)))
    elif pooling=="average":
        model.add(layers.AveragePooling2D((2,2)))
        
    # Second Convolutional Layer
    model.add(DefaultConv2D(filters=64))
    
    # Batch Nomalization
    if batch_normalization==True:
        model.add(layers.BatchNormalization())
    
    # Pooling Layer
    if pooling=="max":
        model.add(layers.MaxPooling2D((2,2)))
    elif pooling=="average":
        model.add(layers.AveragePooling2D((2,2)))
    
    # Flatten Layer
    model.add(layers.Flatten())
    
    # Dense Layer
    model.add(layers.Dense(128, activation=activation))
    
    # Dropout
    if drop_out not None: 
        model.add(layers.Dropout(drop_out))
    
    # Output Layer
    model.add(layers.Dense(3, activation="softmax"))

    
    if optimizer=="adam":
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer =="RMSprop":
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    elif optimizer=="SGD":
        optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate)

    model.compile(optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"]) 
        
    return model

model = KerasClassifier(build_fn=create_model,  verbose=0,  epochs=100, batch_size=20) 

param_distributions = {   "optimizer": ["RMSprop",  "Adam", "SGD"], 
                          "activation": ["relu", "sigmoid"], 
                          "padding": ["SAME", "VALID"],
                          "pooling": ["max", "average"], 
                          "learning_rate": [0.0001, 0.001, 0.01], 
                          "batch_normalization": [True, False], 
                          "drop_out": [None, 0.25, 0.5]
                       # "conv_kernel_size": [(3,3), (5,5)]
                       }

grid = RandomizedSearchCV(estimator=model, param_distributions=param_distributions, n_jobs=-1, 
                          n_iter=50, cv=3, return_train_score=True, scoring='accuracy') # "f1_weighted"

es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=20)

grid_result = grid.fit(X_gadf_train, y_train, class_weight=weights, callbacks=[es])